In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import re
os.chdir(r'\\phont80025us.homeoffice.wal-mart.com\Shares\Audit_Analytics\FY20\Risk Sensing\Auto Care Center\ACC_TBC_Cases')

### 1. Consolidate the information of 117 cases into readable data frame

In [2]:
raw = pd.read_excel('ACC  TBC Cases Data - master doc.xlsx',sheet_name=None)

##### 1.1 Create index indicating the 1st row and last row of each case

In [3]:
df_index = pd.DataFrame(columns=['Case_#','Master_ix','Sub_ix_Start','Sub_ix_End'])
for i in range(117):
    try:
        df_index.loc[i+1,'Case_#'] = i+1
        df_index.loc[i+1,'Master_ix'] = raw[raw.iloc[:,0] ==i+1].index[0]
        df_index.loc[i+1,'Sub_ix_Start'] = raw[raw.iloc[:,0] ==i+1].index[1]
        df_index.loc[i+1,'Sub_ix_End'] = raw[raw.iloc[:,0] ==i+2].index[1]
    except IndexError:
        print(i+1)

df_index.loc[117,'Sub_ix_End'] = len(raw)

df_index['No._Col'] = df_index['Sub_ix_End']-df_index['Sub_ix_Start']

117


##### 1.2 Some cases have different format
- check the differences, then convert to the standard format

In [4]:
# the description table for each case is supposed to have 35 rows
# check which case contains more than 35 rows and fix the exceptions

df_exception = df_index[df_index['No._Col'] != 35]
df_exception

,Case_#,Master_ix,Sub_ix_Start,Sub_ix_End,No._Col
33,33,99,1313,1349,36
39,39,106,1524,1560,36
71,71,140,2645,2681,36
82,82,152,3031,3067,36
95,95,166,3487,3523,36
105,105,177,3838,3874,36
106,106,178,3874,3910,36
114,114,186,4155,4191,36


In [5]:
# See the standardize row names for each case
start = df_index.iloc[0,2]
end = df_index.iloc[0,3]
case_sample= raw.iloc[start:end,[0,5]]  

sd_col = case_sample.iloc[:,0][1:]

In [6]:
# check the difference exist in each exception case
for i in range(len(df_exception)):
    
    start = df_exception.iloc[i,2]    
    end = df_exception.iloc[i,3]
    exp_case = raw.iloc[start:end,[0,5]]
    exp_col = exp_case.iloc[:,0][1:]
    
    print('Case' , df_exception.iloc[i,0],set(exp_col) -set(sd_col ),)
    

('Case', 33, set([nan]))
('Case', 39, set([nan]))
('Case', 71, set([nan]))
('Case', 82, set([nan]))
('Case', 95, set([u'Expert witnesses']))
('Case', 105, set([nan]))
('Case', 106, set([nan]))
('Case', 114, set([nan]))


In [7]:
# Some discription text for above case are split into 2 rows
# so that the name of the discription in a second row is NAN

drop_row = []
for i in range(len(df_exception)):
    try:
        start = df_exception.iloc[i,2]    
        end = df_exception.iloc[i,3]
        exp_case = raw.iloc[start:end,[0,5]]     
        drop_ix  = exp_case[exp_case.iloc[:,0].isnull()].index[0]
        drop_row = drop_row + [drop_ix]
        raw.iloc[drop_ix-1,5] = raw.loc[drop_ix-1,'Unnamed: 5'] + raw.loc[drop_ix,'Unnamed: 5']
    except IndexError:
        drop_ix  = exp_case[exp_case.iloc[:,0] == u'Expert witnesses'].index[0]
        drop_row = drop_row + [drop_ix]
#raw = raw.drop(drop_row) 

In [8]:
df_case_all = pd.DataFrame()
remove_list  = ['Services:','ACC Documentation:','Tire Sale:','Incident:','Lawsuit:',u'Expert witnesses']

for i in range(117):
    start = df_index.iloc[i,2]
    end = df_index.iloc[i,3]
    df_case = raw.iloc[start:end,[0,5]]  
    remove_bool =[(j not in remove_list) and (type(j) != int) for j in df_case.iloc[:,0]]    
    df_case = df_case[remove_bool].T
    
    df_case.columns = df_case.iloc[0,:]
    df_case = df_case.iloc[1:2,:]
    #print(df_case.shape[1],i)
    df_case_all = pd.concat([df_case_all, df_case])

In [9]:
col = [i for i in sd_col if i not in  remove_list]
df_case_all = df_case_all[col]

In [10]:
df_case_all.index = range(len(df_case_all))

### 2. Clean individual columns in the standard format

#####  2.1 Process Service Date, Accident Date, Date Opened, and Date Closed
- Clean various data format
- Create 3 columns:
   
    Days_1stServ_Incd: days inbetween 1st service and the day of incident
    
    Days_1stServ_opd: days inbetween 1st service and case open day
    
    Days_1stServ_cld: days inbetween 1st service and case close day

In [11]:
# among the cases, this case have no info of date of service, set it to September 15, 2008 
print(df_case_all.loc[7,'Date of service'])
df_case_all.loc[7,'Date of service'] = 'September 15, 2008'
datepattern = re.compile("\w* \d*, \d{4}") 

September, 2008 (allegedly)


In [12]:
# find the set of strings which have the Month Day,Year format

for i in df_case_all.index: 

    try: 
        time  = datepattern.findall(df_case_all.loc[i,'Date of service'])
        df_case_all.loc[i,'Date_1stService'] = datetime.strptime(time[0],'%B %d, %Y').strftime('%m/%d/%Y') 
        Incd =  datetime.strptime(df_case_all.loc[i,'Incident date'],'%B %d, %Y')
        df_case_all.loc[i,'Days_1stServ_Incd'] = (Incd - datetime.strptime(time[0],'%B %d, %Y')).days
        
        opd = datetime.strptime(df_case_all.loc[i,'Date Opened'],'%B %d, %Y')        
        df_case_all.loc[i,'Days_1stServ_opd'] = (opd - datetime.strptime(time[0],'%B %d, %Y')).days
        
        cld = datetime.strptime(df_case_all.loc[i,'Date Closed'],'%B %d, %Y')      
        df_case_all.loc[i,'Days_1stServ_cld'] = (cld - datetime.strptime(time[0],'%B %d, %Y')).days
    
    except: 
        #print (df_case_all.loc[i,'Date Closed'])
        df_case_all.loc[i,'Days_1stServ_cld'] = np.nan
    

#####  2.2 Process Store Number and State

In [13]:

# the element under "Store" column is in this format : store number store name
# except for one case: 
# 'ACC – 579 St. Augustine, Florida / TBC – 8138 Daytona Beach, Florida'
def stor_nb(s):
    mo = re.match('.+([0-9])[^0-9]*$',s)
    store_nb = s[0:mo.end(1)]    
    return(int(store_nb))

Store_nb =  []
for i in df_case_all.index:   
    try:
        Store_nb = Store_nb + [stor_nb(df_case_all.loc[i,'Store'])]       
        df_case_all.loc[i,'State'] = df_case_all['Store'][i].split(',')[-1]
        
    except UnicodeEncodeError:
        Store_nb =  Store_nb +[8138]
        df_case_all.loc[i,'State'] = 'Florida'

df_case_all['Store_nb'] = Store_nb

##### 2.3 Process Vehicle Information

In [14]:
ix = df_case_all[df_case_all['Vehicle'].isnull()].index
df_case_all.loc[ix,'Vehicle'] = 'unknown'
df_case_all['Vehicle'] = df_case_all['Vehicle'].apply(lambda x:x.replace(",",""))

In [15]:

# Create columns to store 4 types of Vehicle information:
### 'Mileage'
### 'Veh_Yr'
### 'Veh_Brand'
### 'Veh_Model'

for i in df_case_all.index:

    if '/ Mileage' in df_case_all.loc[i,'Vehicle']:
        
        text1 = df_case_all.loc[i,'Vehicle'].split('/ Mileage')[1]
            
        if ('unknown' in text1)|('not' in text1):
            df_case_all.loc[i,'Mileage'] =  np.nan            
        else: 
            try: df_case_all.loc[i,'Mileage'] = re.findall(r'\d+',text1)[0]
            except IndexError: df_case_all.loc[i,'Mileage'] =  np.nan
                
        text2 = df_case_all.loc[i,'Vehicle'].split('/ Mileage')[0]        
        try: 
            df_case_all.loc[i,'Veh_Yr'] = int(re.findall(r'\d+',text2)[0])
            text3 = text2[5:]
            df_case_all.loc[i,'Veh_Brand'] = text3.split(' ',1)[0]
            df_case_all.loc[i,'Veh_Model'] = text3.split(' ',1)[1]
            
        except IndexError: 
            df_case_all.loc[i,'Veh_Yr'] = np.nan 
            df_case_all.loc[i,'Veh_Brand'] = text2.split(' ',1)[0]
            df_case_all.loc[i,'Veh_Model'] = text2.split(' ',1)[1]
                        
        
    else: 
        
        df_case_all.loc[i,'Mileage'] =  np.nan
        Veh = df_case_all.loc[i,'Vehicle']
        if ('unknown' in Veh)|('not' in Veh)|('Unknown' in Veh):
            df_case_all.loc[i,'Veh_Yr'] = np.nan
            df_case_all.loc[i,'Veh_Brand'] = np.nan
            df_case_all.loc[i,'Veh_Model'] = np.nan
        else:
            try:
                df_case_all.loc[i,'Veh_Yr'] = re.findall(r'\d+',Veh)[0]
                text4 = df_case_all.loc[i,'Vehicle'][5:]
                df_case_all.loc[i,'Veh_Brand'] = text4.split(' ',1)[0]
                df_case_all.loc[i,'Veh_Model'] = text4.split(' ',1)[1]
            except IndexError:            
                df_case_all.loc[i,'Veh_Brand'] = Veh.split(' ',1)[0]
                df_case_all.loc[i,'Veh_Model'] = Veh.split(' ',1)[1]
        

##### 2.4 Calculate service time

- if the cases have multiple services, keep the longest service time


In [16]:
ix = df_case_all[df_case_all['Arrival & service complete times'].isnull()].index
df_case_all.loc[ix,'Arrival & service complete times'] = 'unknown'
for i in df_case_all.index:
    #print(i)
    text = df_case_all.loc[i,'Arrival & service complete times']
    if  ('Unknown' in text)|('unknown' in text)|('no' in text)|('not' in text):
        df_case_all.loc[i,'Sev_Time'] = np.nan
   
    else:
        
        timepattern = re.compile("\d*:\d{2}")
        time  = timepattern.findall(text)
        APM = re.findall("AM|PM", text)
        
        for j in range(len(time)):
            if (APM[j] == 'PM') & (int(time[j].split(":")[0])<12):
                time[j] = (int(time[j].split(":")[0])+12) * 60 + int(time[j].split(":")[1])
            else: time[j] = int(time[j].split(":")[0]) * 60  + int(time[j].split(":")[1])
        
        temp_max = time[1]-time[0]
        for k in range(1,len(time)/2):
            if time[2*k+1]-time[2*k] > temp_max: temp_max = time[2*k+1]-time[2*k]
        df_case_all.loc[i,'Sev_Time'] = temp_max


##### 2.5 Create dummy columns for various service
- Tire Installation
- Tire Rotation
- Tire Repair
- Oil Change
- Battery
- Others

In [17]:
df_case_all['Tire Installation'] = 0
df_case_all['Tire Rotation'] = 0
df_case_all['Tire Repair'] = 0
df_case_all['Oil Change'] = 0 
df_case_all['Battery'] = 0
#df_case_all['Others'] = 0



bool1 = ['install' in  df_case_all.loc[i,'Services/Products provided'].lower() for i in df_case_all.index]
ix1   = df_case_all[bool1].index
df_case_all.loc[ix1, 'Tire Installation'] = 1

bool2 = ['rotat' in  df_case_all.loc[i,'Services/Products provided'].lower() for i in df_case_all.index]
ix2   = df_case_all[bool2].index
df_case_all.loc[ix2, 'Tire Rotation'] = 1

bool3 = ['repair' in  df_case_all.loc[i,'Services/Products provided'].lower() for i in df_case_all.index]
ix3   = df_case_all[bool3].index
df_case_all.loc[ix3, 'Tire Repair'] = 1

bool4 = ['oil change' in  df_case_all.loc[i,'Services/Products provided'].lower() for i in df_case_all.index]
ix4   = df_case_all[bool4].index
df_case_all.loc[ix4, 'Oil Change'] = 1

bool5 = ['battery' in  df_case_all.loc[i,'Services/Products provided'].lower() for i in df_case_all.index]
ix5   = df_case_all[bool5].index
df_case_all.loc[ix5, 'Battery'] = 1




##### 2.6 Check how many associates were involved in the provided services and if QC involved


In [18]:
ix = df_case_all[df_case_all['Service technicians'].isnull()].index
df_case_all.loc[ix,'Service technicians'] = 'unknown'


df_case_all['Count_Ser_Asso'] = np.nan
df_case_all['QC?'] = 0

assopat = re.compile('\d+') 
for i in df_case_all.index:
    
    text = df_case_all.loc[i,'Service technicians'].lower()
    #print(i,text)
    if ('unknown' in text)|('not' in text)|('none' in text):
        df_case_all.loc[i,'Count_Ser_Asso'] = -1
        df_case_all.loc[i,'QC?'] = -1
    else:
        # check if QC involved
        if 'quality' in text:
            df_case_all.loc[i,'QC?'] = 1
        else: df_case_all.loc[i,'QC?'] = 0
        
        # if there are multiple times of services, used the info of 1st service        
        ser_date =  datepattern.findall(text)
        
        if len(ser_date)>1:
            begin = text.find(ser_date[0])+ len(ser_date[0])
            cut = text.find(ser_date[1])-1
            text = text[begin:cut]            
            
            df_case_all.loc[i,'Count_Ser_Asso'] = len(assopat.findall(text))
            
        elif len(ser_date)==1:
            begin = text.find(ser_date[0])+ len(ser_date[0])
            text = text[begin:] 
            df_case_all.loc[i,'Count_Ser_Asso'] = len(assopat.findall(text))
        
        else:
            
            if len(assopat.findall(text))>0:
                df_case_all.loc[i,'Count_Ser_Asso'] = len(assopat.findall(text))
            else: 
                
                text = text.replace('/greeter',"")
                df_case_all.loc[i,'Count_Ser_Asso'] = text.count('/')+text.count('and ')+1
                
            
            

##### 2.7 Clean up Defense costs and Payout


In [19]:
# 
money = re.compile(r"\$\d*")
for i in df_case_all.index:
    #Clean Payout
    try:
        pay  = money.findall(df_case_all.loc[i,'Payout'].replace(",",""))       
        df_case_all.loc[i,'Payout'] = int(pay[0].replace("$",""))
        
    except AttributeError:  df_case_all.loc[i,'Payout'] = df_case_all.loc[i,'Payout']
    except IndexError: df_case_all.loc[i,'Payout']  = 0

    #Clean Defense costs
    try:
        cost  = money.findall(df_case_all.loc[i,'Defense costs'].replace(",","")) 
        #print(i,df_case_all.loc[i,'Defense costs'],cost)
        df_case_all.loc[i,'Defense costs'] = int(cost[0].replace("$",""))
    except AttributeError: df_case_all.loc[i,'Defense costs'] = df_case_all.loc[i,'Defense costs'] 
    except IndexError: df_case_all.loc[i,'Defense costs']  = 0       
   

##### 2.8 Multiple cases will be initiated for 1 incident
- Keep one line of the incident and sum up the payout and cost

In [20]:
# Duplicated Cases

df_dupl_case  = df_case_all[['related' in i for i in df_case_all['Matter number']]].sort_values(by=['Customer'])

for i in df_dupl_case['Customer'].unique():
    ix  = df_dupl_case[df_dupl_case['Customer'] == i].index
    df_dupl_case.loc[ix,'Defense costs'] = sum(df_dupl_case.loc[ix,'Defense costs'])
    df_dupl_case.loc[ix,'Payout'] = sum(df_dupl_case.loc[ix,'Payout'])

df_sub = df_dupl_case.drop_duplicates(['Customer'])

In [21]:
df_no_dupl = df_case_all[[ i not in df_dupl_case.index for i in  df_case_all.index]]

In [22]:
df_case_clean = pd.concat([df_no_dupl,df_sub])

In [23]:
df_case_clean.to_csv('acc_cleaned.csv',index = False,encoding='utf_8_sig')